In [16]:
import pandas as pd 
import string
import numpy as np
import sklearn as sk
import tensorflow as tf
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from transformers import AutoTokenizer, TFBertModel 

In [17]:
cwd = os.getcwd()
datapath = os.path.join(cwd, 'dataset')

In [18]:
df = pd.read_excel(os.path.join(datapath, 'df_new.xlsx'))
# df['berita'] = df['berita'].apply(lambda x : " ".join(eval(x)))
# df['berita'] = df['berita'].apply(lambda x : eval(x))
df = df[['cleaned', 'label']]
df

,cleaned,label
0,PSI Sebut Banding Pemecatan Viani Limardi Dito...,0
1,"Ditawarkan Koalisi dengan Gerindra , PKS Bilan...",0
2,DPR - Pemerintah - KPU Sepakat Honor Petugas T...,0
3,"Megawati dan Langkah Politiknya sebagai "" Quee...",0
4,"Dana Pemilu 2024 Capai Rp 110 , 4 Triliun , Jo...",0
...,...,...
14723,postingan isi jelas sekaligus klarifikasi buat...,1
14724,mardani kata foto pegang kaos warna hitam tuli...,1
14725,foto tribunnewscom debunk isi buah nyata klari...,1
14726,foto twitter jbillinson debunk isi bantah lege...,1


In [19]:
df.cleaned.isna().value_counts()

cleaned
False    14727
True         1
Name: count, dtype: int64

In [4]:
X, y = df['cleaned'], df['label']
X.shape, y.shape

((22556,), (22556,))

In [5]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42,
                                     stratify=df['label'])

In [6]:
df_train.shape, df_test.shape

((15789, 2), (6767, 2))

In [7]:
model = 'indobenchmark/indobert-base-p1'

tokenizer = AutoTokenizer.from_pretrained(model)
bert = TFBertModel.from_pretrained(model)

max_len = 70

Some layers from the model checkpoint at indobenchmark/indobert-base-p1 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [10]:
df_train.cleaned.isna().value_counts()

cleaned
False    13056
True      2733
Name: count, dtype: int64

In [9]:
X_train = tokenizer(
    text=df_train['cleaned'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

X_test = tokenizer(
    text=df_test['cleaned'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids, attention_mask = input_mask)[0]

embeddings = bert(input_ids, attention_mask = input_mask)[0] # 0 = last hidden state, 1 = poller_output
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation='relu')(out)

y = Dense(2)(out)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

optimizer = Adam(
    learning_rate=5e-05, # HF recommendation
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0
)

loss = CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy('balanced_accuracy')

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric
)

history = model.fit(
    x = {'input_ids':X_train['input_ids'], 'attention_mask':X_train['attention_mask']},
    y = to_categorical(df_train['label']),
    validation_data = ({'input_ids':X_test['input_ids'], 'attention_mask':X_test['attention_mask']},
                        to_categorical(df_test['label'])),
    epochs=1,
    batch_size=32
)

In [ ]:
from sklearn.metrics import classification_report

predicted = model.predict({'input_ids': X_test['input_ids'], 'attention_mask': X_test['attention_mask']})
y_predicted = np.argmax(predicted, axis=1)


In [ ]:
print(classification_report(df_test['label'], y_predicted))